In [3]:
import datetime as dt
import os
import requests
import time
from urllib import parse
from glob import glob

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm, trange


base_url = "https://namu.wiki/w/"
logging_dir = "/home/nsw/datasets/namuwiki_scrapping_logs/"

# beautifulsoup

## graph

In [31]:
step = len(glob(logging_dir + ".xlsx"))
log_cols = ["parents", "key", "use"]
graph = dict()

In [32]:
if not os.path.isfile(logging_dir + "result_step_00.xlsx"):
    pd.DataFrame([("root", "우왁굳", 1)], columns=log_cols).to_excel(logging_dir + f"result_step_{step:02d}.xlsx", index=None)

In [27]:
def make_node(title: str, children: list):
    return {
        "title": title,
        "url": parse.quote(base_url + title),
        "children": [ch[1] for ch in children],
    }

In [45]:
key_df = pd.read_excel(logging_dir + f"result_step_{step:02d}.xlsx")
for _, row in key_df.iterrows():
    if pd.isna(row['use']):
        graph[row['parents']]['children'].remove(row['key'])

keys = key_df.dropna().key.to_list()

# keys = pd.read_excel(logging_dir + f"result_step_{step:02d}.xlsx").key.dropna().to_list()
step += 1
new_keys = []
for key in tqdm(keys):
    if key in graph:
        # print(key, ": continue")
        continue
    soup = BeautifulSoup(requests.get(base_url + key).text, 'html.parser')

    index = soup.select_one("div.toc-indent")
    if index == None:
        children = []
    else:
        children = [
            (key, parse.unquote(link.get_attribute_list("href")[0].split("/", 2)[-1]), 1)
            for link in index.select("a.wiki-link-internal")
        ]

    node = make_node(key, children)
    graph[key] = node
    # print(key, children)

    new_keys += children

# new_keys
pd.DataFrame(new_keys, columns=log_cols).to_excel(logging_dir + f"result_step_{step:02d}.xlsx", index=None)

  0%|          | 0/101 [00:00<?, ?it/s]

In [25]:
graph.keys()

dict_keys(['우왁굳', '우왁굳/방송 역사', '왁두', '우왁굳/특징', '우왁굳/별명 및 밈', '우왁굳/콘텐츠', '우왁굳/플레이한 게임', '침팬치', '왁물원', '우왁굳TV', '왁카데미', '우왁굳/콘텐츠/고정 멤버', '이세계아이돌', '왁타버스 중간계', '우왁굳/논란 및 사건 사고', '우왁굳/어록', '오뱅계르크', '우왁굳/연도별 행적', '아프리카TV', '왁두/파생형', '우왁굳/콘텐츠/게임', '우왁굳/콘텐츠/비게임', '왁타버스', '고멤 놀이터', '왁물원/이세계아이돌 게시판', '왁물원/인력사무소', '우왁굳/콘텐츠/비게임/연말 공모전', '왁물원/규정', '네이버 카페', '왁물원/역사', '개복어', '노돌리', '레알마틴', '렘쨩', '로복', '무결', '조마문', '조매력', '천양', '크헐헐', '도그홀', '두칠', '바나나(인터넷 방송인)', '진호(인터넷 방송인)', '함군', '겜스터', '휘용', '고니잠', '주르르', '우왁굳/콘텐츠/고정 멤버/0기', '왁타버스 고정 멤버', '고멤 아카데미', '이세계아이돌/멤버 간 케미', '이세계아이돌/오디션', '이세계아이돌/활동', '이세계아이돌/공연 및 행사', '이세계아이돌/음반 목록', '이세계아이돌/오리지널 곡', '이세계아이돌/커버 곡', '마법소녀 이세계아이돌', '차원을 넘어 이세계아이돌', '이파리', '이세계아이돌/응원법', '이세계아이돌/굿즈', '이세계아이돌/이모티콘', '이세계아이돌/월드', '이세계아이돌/여담', '2020년 유튜버 뒷광고 사건'])

In [30]:
graph['우왁굳/논란 및 사건 사고']['children']

[('우왁굳/논란 및 사건 사고', 'One Hour One Life'),
 ('우왁굳/논란 및 사건 사고', '그리핑'),
 ('우왁굳/논란 및 사건 사고', '2020년 유튜버 뒷광고 사건'),
 ('우왁굳/논란 및 사건 사고', '왁타버스')]

## Tree  
`분류:` 항목 참고한 트리구조(그래프x)

In [76]:
def make_tree_node(
    title: str, 
    parents: str, 
    subcategory: list, 
    subdocs: list, 
    subform: list
):
    return {
        "title": title,
        "url": parse.quote(base_url + "분류:" + title),
        "parents": parents,
        "subcategory": subcategory,
        "subdocs": subdocs,
        "subform": subform,
    }

In [80]:
tree = {}
next_keys = [("root", "우왁굳")]

while len(next_keys) > 0:
    keys = next_keys
    next_keys = []

    for parents, key in tqdm(keys, leave=False):
        if key in tree:
            continue
        soup = BeautifulSoup(requests.get(base_url + "분류:" + key).text, 'html.parser')

        subcategory = soup.select_one("#category-분류")
        subdocs = soup.select_one("#category-문서")
        subform = soup.select_one("#category-틀")
        
        subcategory = [] if subcategory is None else [tag.get_attribute_list("title")[0].split(":", 1)[-1] for tag in subcategory.select("a")]
        subdocs = [] if subdocs is None else [tag.get_attribute_list("title")[0] for tag in subdocs.select("a")]
        subform = [] if subform is None else [tag.get_attribute_list("title")[0].split(":", 1)[-1] for tag in subform.select("a")]

        node = make_tree_node(key, parents, subcategory, subdocs, subform)
        tree[key] = node
        # print(key, children)
    
        next_keys += [(key, sc) for sc in subcategory]

    print(next_keys)

  0%|          | 0/1 [00:00<?, ?it/s]

[('우왁굳', '왁물원'), ('우왁굳', '우왁굳/밈'), ('우왁굳', '우왁굳/콘텐츠'), ('우왁굳', '파일/우왁굳')]


  0%|          | 0/4 [00:00<?, ?it/s]

[('우왁굳/콘텐츠', '왁타버스'), ('우왁굳/콘텐츠', '우왁굳/콘텐츠/게임'), ('우왁굳/콘텐츠', '우왁굳/콘텐츠/고정 멤버'), ('파일/우왁굳', '파일/고멤 가요제'), ('파일/우왁굳', '파일/왁타버스')]


  0%|          | 0/5 [00:00<?, ?it/s]

...

[('고멤 가요제', '고멤 가요제/노래'), ('고멤 가요제', '고멤 가요제/팀'), ('고멤 가요제', '파일/고멤 가요제'), ('고멤 아카데미', '고멤 아카데미/소속 멤버'), ('고멤 아카데미', '고멤 아카데미/오리지널 곡'), ('고멤 아카데미', '파일/고멤 아카데미'), ('왁타버스 고정 멤버', '왁타버스 고정 멤버/소속 멤버'), ('왁타버스 고정 멤버', '왁타버스 고정 멤버/시즌'), ('왁타버스 고정 멤버', '왁타버스 고정 멤버/오리지널 곡'), ('왁타버스 고정 멤버', '파일/왁타버스 고정 멤버'), ('왁타버스/오리지널 곡', '고멤 아카데미/오리지널 곡'), ('왁타버스/오리지널 곡', '왁타버스 고정 멤버/오리지널 곡'), ('왁타버스/오리지널 곡', '이세계아이돌/오리지널 곡'), ('이세계아이돌', '이세계아이돌/2차 창작'), ('이세계아이돌', '고세구'), ('이세계아이돌', '이세계아이돌/공연 및 행사'), ('이세계아이돌', '둘러보기 틀/이세계아이돌'), ('이세계아이돌', '릴파'), ('이세계아이돌', '마법소녀 이세계아이돌'), ('이세계아이돌', '비챤'), ('이세계아이돌', '아이네'), ('이세계아이돌', '이세계아이돌/오리지널 곡'), ('이세계아이돌', '이세계아이돌/음반'), ('이세계아이돌', '이파리'), ('이세계아이돌', '주르르'), ('이세계아이돌', '징버거'), ('이세계아이돌', '차원을 넘어 이세계아이돌'), ('이세계아이돌', '파일/이세계아이돌'), ('우왁굳/콘텐츠/게임/VR챗', '2023 왁타버스 뮤직 어워즈'), ('우왁굳/콘텐츠/게임/VR챗', '고멤 가요제'), ('파일/이세계아이돌', '파일/고세구'), ('파일/이세계아이돌', '파일/릴파'), ('파일/이세계아이돌', '파일/비챤'), ('파일/이세계아이돌', '파일/아이네'), ('파일/이세계아이돌', '파일/주르르'), ('파일/이세계아이돌', '파일/징버거')]


  0%|          | 0/36 [00:00<?, ?it/s]

[('왁타버스 고정 멤버/오리지널 곡', '고멤 가요제/노래'), ('고세구', 'TOUR'), ('고세구', '파일/고세구'), ('이세계아이돌/공연 및 행사', 'TOUR'), ('릴파', '파일/릴파'), ('마법소녀 이세계아이돌', '마법소녀 이세계아이돌/등장인물'), ('비챤', '파일/비챤'), ('아이네', '파일/아이네'), ('주르르', '파일/주르르'), ('징버거', '파일/징버거'), ('차원을 넘어 이세계아이돌', '차원을 넘어 이세계아이돌/등장인물')]


  0%|          | 0/11 [00:00<?, ?it/s]

[]


## draw graph

In [70]:
plant_uml = (
    "@startuml\n"
    "skinparam linetype polyline\n"
    "skinparam linetype ortho\n\n"
)

In [71]:
nodes = set(list(graph.keys()) + sum([node['children'] for node in graph.values()], []))
nodes_dict = {node: f"node_{i}" for i, node in enumerate(nodes)}

for node, name in nodes_dict.items():
    plant_uml += f'rectangle "{node}" as {name}\n'

In [72]:
for key, value in graph.items():
    for ch in value['children']:
        plant_uml += f"{nodes_dict[key]} --> {nodes_dict[ch]}\n"

plant_uml += "@enduml"

In [3]:
driver = webdriver.Chrome()
driver.get(root_url)

In [18]:
index = driver.find_element(By.CSS_SELECTOR, "#app > main > div.i8lJknch > div.DyX8bvYU > article > div.RUUPhBBO > div > div:nth-child(2) > div > div > div:nth-child(7) > div.VMXJxT\+c")
a_tag_list = index.find_elements(By.CSS_SELECTOR, "a.kbELVgKG")
for a in a_tag_list:
    print(a.text, " | ", parse.unquote(a.get_property("href").split("/", 4)[-1]))

방송 역사  |  우왁굳/방송 역사
마스코트 캐릭터  |  왁두
특징  |  우왁굳/특징
별명 및 밈  |  우왁굳/별명 및 밈
콘텐츠  |  우왁굳/콘텐츠
플레이한 게임  |  우왁굳/플레이한 게임
팬덤  |  침팬치
팬카페  |  왁물원
우왁굳TV  |  우왁굳TV
왁카데미  |  왁카데미
고정 멤버  |  우왁굳/콘텐츠/고정 멤버
이세계아이돌  |  이세계아이돌
왁타버스 중간계  |  왁타버스 중간계
논란 및 사건 사고  |  우왁굳/논란 및 사건 사고
어록  |  우왁굳/어록
굿즈  |  오뱅계르크


In [19]:
driver.quit()